In [36]:
from feature_selection import movie
from feature_selection import scraper_rotten_tomatoes
import os
import pickle
import seaborn as sns
from multiprocessing import Pool
from datetime import datetime
import numpy as np
import operator

sns.set(color_codes=True)

In [2]:
movie_dict = movie.get_info_with_localname('Black Panther(2018)')
print(movie_dict['summary']['Genre'])
budget = movie_dict['summary']['Budget']
revenue = movie_dict['international']['Worldwide Box Office'][0]['Revenue']
ROI = (revenue - budget) / budget
print(ROI)

Action
5.667347265


In [22]:
def is_released(movie_name, movie_infos):
    if 'Domestic Releases' not in movie_infos[movie_name]['summary']:
        return False
    elif len(movie_infos[movie_name]['summary']['Domestic Releases'].keys()) == 0:
        return False
    else:
        release = movie_infos[movie_name]['summary']['Domestic Releases']
        if 'Wide' in release and release['Wide'] > datetime.now():
            return False
        elif 'IMAX' in release and release['IMAX'] > datetime.now():
            return False
        elif 'Limited' in release and release['Limited'] > datetime.now():
            return False
        else:
            return True

# get the names of all the movie
def get_movie_names_all():
    with open('movie_info.pkl', 'rb') as f:
        movie_infos = pickle.load(f)
    count = 0
    movie_names = list(movie_infos.keys())
    movie_names = [movie_name for movie_name in movie_names if is_released(movie_name, movie_infos)]
    return movie_names

print(get_movie_names_all()[:10])

['Avatar(2009)', 'Pirates of the Caribbean: On Stranger Tides(2011)', 'Avengers: Age of Ultron(2015)', 'Star Wars Ep. VII: The Force Awakens(2015)', 'Pirates of the Caribbean: At World’s End(2007)', 'Justice League(2017)', 'Spectre(2015)', 'The Dark Knight Rises(2012)', 'The Lone Ranger(2013)', 'John Carter(2012)']


In [42]:
# calculate the ROI of a movie
def get_ROI(movie_name, all_dict):
    movie_dict = all_dict[movie_name]
    budget = movie_dict['summary']['Budget']
    if len(movie_dict['international']['Worldwide Box Office']) != 0:
        revenue = movie_dict['international']['Worldwide Box Office'][0]['Revenue']
    elif len(movie_dict['international']['International Box Office']) != 0:
        revenue = movie_dict['international']['International Box Office'][0]['Revenue']
    elif len(movie_dict['box_office']['demostic']) != 0:
        revenue = movie_dict['box_office']['demostic'][0]['Revenue']
    else:
        return None
    if movie_name == 'Deep Throat(1972)':
        print(revenue, budget)
    ROI = (revenue - budget) / budget
    return ROI

def get_ROIs():
    movie_names = get_movie_names_all()
    print(movie_names[:10])
    
    with open('movie_info.pkl', 'rb') as f:
        movie_dict = pickle.load(f)
    
    output = []
    for movie_name in movie_names:
        ROI = get_ROI(movie_name, movie_dict)
        if ROI is not None:
            output.append( (movie_name, ROI) )
    
    output.sort(key=operator.itemgetter(1))
    output.reverse()
    return output

ROIs = get_ROIs()
print(ROIs[:100])

['Avatar(2009)', 'Pirates of the Caribbean: On Stranger Tides(2011)', 'Avengers: Age of Ultron(2015)', 'Star Wars Ep. VII: The Force Awakens(2015)', 'Pirates of the Caribbean: At World’s End(2007)', 'Justice League(2017)', 'Spectre(2015)', 'The Dark Knight Rises(2012)', 'The Lone Ranger(2013)', 'John Carter(2012)']
45000000 25000
[('Deep Throat(1972)', 1799.0), ('Mad Max(1980)', 497.75), ('Paranormal Activity(2009)', 430.51785333333333), ('The Gallows(2015)', 416.53881), ('The Blair Witch Project(1999)', 412.8333333333333), ('Super Size Me(2004)', 341.05858461538463), ('Bambi(1942)', 311.35431235431236), ('El Mariachi(1993)', 290.704), ('Night of the Living Dead(1968)', 262.9216140350877), ('Rocky(1976)', 224.0), ('Halloween(1978)', 214.3846153846154), ('The Brothers McMullen(1995)', 207.53012), ('The Texas Chainsaw Massacre(1974)', 188.8031357142857), ('American Graffiti(1973)', 179.18018018018017), ('My Date With Drew(2005)', 163.58272727272728), ('Once(2007)', 154.49371333333335), (

In [43]:
# visualize the ROI distributions
ROI_to_visualize = np.array([ROI[1] for ROI in ROIs])
print(ROI_to_visualize)
sns.distplot(ROI_to_visualize)

5051
[  1.79900000e+03   4.97750000e+02   4.30517853e+02 ...,  -9.99874545e-01
  -9.99974000e-01  -9.99974900e-01]
